In [1]:
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

In [2]:
input_ = {
    "model_id": "20231026000000",
    "start_time":  "2023-09-01 00:00:00",
    "end_time":    "2023-10-10 00:00:00",
    "model_type": 0
}
start_time = input_["start_time"]
end_time = input_["end_time"]

In [3]:
start_time = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')
end_time = datetime.strptime(end_time, '%Y-%m-%d %H:%M:%S')

In [4]:
df_raw = pd.read_excel("data/train/train.xlsx", sheet_name = "simulation", header = [0, 1, 2, 3])
df_raw.iloc[:, 0] = pd.to_datetime(df_raw.iloc[:, 0])
df_raw = df_raw[(start_time <= df_raw.iloc[:, 0]) & (df_raw.iloc[:, 0] <= end_time)]
print(f"length: {len(df_raw)}")
print(f"columns: {df_raw.shape[1]}")
df_raw.head(3)

length: 1200
columns: 91


Unnamed: 0_level_0              A0                              \
    Unnamed: 0_level_1              左邊                               
    Unnamed: 0_level_2 直徑(1)\n圓(A0.左邊)                               
           update_time              NV      AV     DV     UT LT ER   
0  2023-09-20 17:53:00              98  98.019  0.019  0.022  0  0   
1  2023-09-20 19:32:00              98  98.015  0.015  0.022  0  0   
2  2023-09-21 11:18:00              98  98.012  0.012  0.022  0  0   

                                     ...                         A180         \
                                     ...                          左右邊          
  Y距離(7)\n距離(A0.左邊)                  ... Y距離(6)\n距離(A180.左邊)(A180.右邊)          
                 NV       AV     DV  ...                           DV     UT   
0               100  100.011  0.011  ...                        0.008  0.019   
1               100  100.000  0.000  ...                        0.011  0.019   
2               100   99.997 -0.003  ...                       -0.008  0.019   

                   None                                     
                   None                                     
            XY距離(1)\n距離                                     
  LT     ER          NV       AV     DV    UT    LT     ER  
0  0  0.000       240.3  240.301  0.001  0.05 -0.05  0.000  
1  0  0.000       240.3  240.321  0.021  0.05 -0.05  0.000  
2  0 -0.008       240.3  240.242 -0.058  0.05 -0.05 -0.008  

[3 rows x 91 columns]

In [5]:
# set columns for each models
model_cols = {
    "model1": {
        "primary":   ["Z距離(8)\n距離(A0.右邊)"],
        "secondary": ["Z距離(7)\n距離(A0.左邊)"]
    },
    "model2": {
        "primary":   ["Y距離(9)\n距離(A180.左邊)"],
        "secondary": ["Y距離(10)\n距離(A180.右邊)"]
    },
    "model3": {
        "primary":   ["Y距離(6)\n距離(A180.左邊)(A180.右邊)"],
        "secondary": ["Y距離(9)\n距離(A180.左邊)", "Y距離(10)\n距離(A180.右邊)"]
    },
    "model4": {
        "primary":   ["XY距離(1)\n距離"],
        "secondary": ["Y距離(7)\n距離(A0.左邊)",   "Y距離(8)\n距離(A0.右邊)",    "Y距離(5)\n距離(A0.左邊)(A0.右邊)",
                      "Y距離(9)\n距離(A180.左邊)", "Y距離(10)\n距離(A180.右邊)", "Y距離(6)\n距離(A180.左邊)(A180.右邊)"]
    },
}

In [6]:
# split data by model
dfs = []
for model, values in model_cols.items():
    cols = values["secondary"] + values["primary"]
    cols = [i for i in df_raw.columns if (i[2] in cols)]
    dfs.append(df_raw[cols])

In [7]:
# deal with constant value
constant_summary = []
for i in range(len(dfs)):
    drop_flag = (dfs[i].nunique() == 1)
    keep_cols = dfs[i].columns[~drop_flag]
    constant_summary.append({
        "init":  dfs[i].shape[1], 
        "drop":  drop_flag.sum(), 
        "keep":  len(keep_cols)
        })
    dfs[i] = dfs[i][keep_cols]
constant_summary = pd.DataFrame(constant_summary, index = ["model1", "model2", "model3", "model4"])
constant_summary

,init,drop,keep
model1,12,7,5
model2,12,6,6
model3,18,9,9
model4,42,23,19


In [8]:
# generate target
target_counts = pd.DataFrame()
target_summary = []
for i, df in enumerate(dfs):
    DV_col = [i for i in df.columns if i[3] == "DV"]
    DV_sum = df[DV_col[:-1]].sum(axis = 1).round(4)
    lower = (DV_sum.median() - DV_sum.std()/2).round(4)
    upper = (DV_sum.median() + DV_sum.std()/2).round(4)

    bins = [float('-inf'), lower, upper, float('inf')]
    DV_target = pd.cut(DV_sum, bins, labels = [0, 1, 2]) # 0: -0.001, 1: 0, 2: +0.001
    # df["sum"] = DV_sum
    df["target"] = DV_target
    
    # record target counts
    counts = df["target"].value_counts().sort_index()
    counts = counts.rename(f"model{i+1}")
    target_counts = pd.concat([target_counts, counts], axis = 1)

    # record target summary
    target_summary.append({
        "median": DV_sum.median().round(4),
        "std": DV_sum.std().round(4),
        "lower": lower, 
        "upper": upper
        })
target_counts = target_counts.T
target_summary = pd.DataFrame(target_summary, index = ["model1", "model2", "model3", "model4"])
target_summary

,median,std,lower,upper
model1,0.002,0.0061,-0.0011,0.0051
model2,0.001,0.0064,-0.0022,0.0042
model3,0.000,0.0091,-0.0045,0.0045
model4,0.019,0.0156,0.0112,0.0268


In [9]:
# rename columns
for df in dfs:
    features = []
    for cols in df.columns:
        if cols[0] in ["sum", "target"]:
            features.append(cols[0])
        else:
            features.append(cols[2] + "_" + cols[3])
    df.columns = features

In [10]:
df.head(3)

,Y距離(7)\n距離(A0.左邊)_AV,Y距離(7)\n距離(A0.左邊)_DV,Y距離(7)\n距離(A0.左邊)_ER,Y距離(8)\n距離(A0.右邊)_AV,Y距離(8)\n距離(A0.右邊)_DV,Y距離(5)\n距離(A0.左邊)(A0.右邊)_AV,Y距離(5)\n距離(A0.左邊)(A0.右邊)_DV,Y距離(9)\n距離(A180.左邊)_AV,Y距離(9)\n距離(A180.左邊)_DV,Y距離(9)\n距離(A180.左邊)_ER,Y距離(10)\n距離(A180.右邊)_AV,Y距離(10)\n距離(A180.右邊)_DV,Y距離(10)\n距離(A180.右邊)_ER,Y距離(6)\n距離(A180.左邊)(A180.右邊)_AV,Y距離(6)\n距離(A180.左邊)(A180.右邊)_DV,Y距離(6)\n距離(A180.左邊)(A180.右邊)_ER,XY距離(1)\n距離_AV,XY距離(1)\n距離_DV,XY距離(1)\n距離_ER,target
0,100.011,0.011,0.001,99.991,-0.009,66.012,0.012,99.990,-0.010,0.0,100.010,0.010,0.0,66.008,0.008,0.000,240.301,0.001,0.000,1
1,100.000,0.000,0.000,99.999,-0.001,66.012,0.012,99.999,-0.001,0.0,99.995,-0.005,0.0,66.011,0.011,0.000,240.321,0.021,0.000,1
2,99.997,-0.003,0.000,100.001,0.001,66.002,0.002,99.996,-0.004,0.0,99.998,-0.002,0.0,65.992,-0.008,-0.008,240.242,-0.058,-0.008,0


In [11]:
with pd.ExcelWriter("data/train/train_split.xlsx", engine = "openpyxl", mode = "a", if_sheet_exists = "replace") as writer: # , mode = "a", if_sheet_exists = "replace"
    # constant_summary.to_excel(writer, sheet_name = "constant_summary")
    # target_counts.to_excel(writer, sheet_name = "target_counts")
    # target_summary.to_excel(writer, sheet_name = "target_summary")

    for i, df in enumerate(dfs):
        df.to_excel(writer, sheet_name = f"model{i+1}", index = False)